In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [9]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document


In [10]:
kendraIndexId = "063e46f7-1953-4503-a46c-72aa1ddf826f"

kendra_retriever = AmazonKendraRetriever(
    index_id= kendraIndexId
)


In [11]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [12]:
llm2 = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-instruct-bf16-2023-06-23-19-34-33-102",
#endpoint_name="hf-llm-falcon-7b-bf16-2023-06-24-20-08-14-262",
#endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 400
              }},
region_name="us-east-1",
content_handler=content_handler
)


In [ ]:
llm_query= "my airpods are not connecting to the iphone, how do I fix that "

prompt_template = """
  {context}
  >>QUESTION<<: {question}
  >>ANSWER<<:"""
  
PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  
chain = load_qa_chain(llm=llm2, prompt=PROMPT)

  

In [ ]:
docs = kendra_retriever.get_relevant_documents(llm_query)
print(docs)

In [ ]:
output = chain({"input_documents":docs, "question": llm_query}, return_only_outputs=False)
print(output['output_text'])
